<a href="https://colab.research.google.com/github/Alton01/ml-vtm-pytorch-ciphar10/blob/main/cifar_10VisionTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import random
import matplotlib.pyplot as plt

In [7]:
torch.__version__

'2.6.0+cu124'

In [9]:
torchvision.__version__

'0.21.0+cu124'

In [11]:
## 2. setup device agnostic code

In [12]:
torch.cuda.is_available()

True

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [14]:
print(f"Using Device: {device}")

Using Device: cuda


In [16]:
## 3. Set the seed

In [17]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
random.seed(42)

In [19]:
3e-4

0.0003

In [18]:
## 4. Setting the hyperparameters

In [20]:
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 3e-4
PATCH_SIZE = 4
NUM_CLASSES = 10
IMAGE_SIZE = 32
CHANNELS = 3
EMBED_DIM = 256
NUM_HEADS = 8
DEPTH = 6
MLP_DIM = 512
DROP_RATE = 0.1

In [ ]:
## 5.Define image transformations operation

In [21]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
    #helps the model to converge faster
    # helps to make the numerical computation stable
])

In [22]:
## 6. Getting a dataset

In [23]:
train_dataset = datasets.CIFAR10(root="data",
                                 train=True,
                                 download=True,
                                 transform=transform)

100%|██████████| 170M/170M [00:15<00:00, 11.1MB/s]


In [24]:
test_dataset = datasets.CIFAR10(root="data",
                                train=False,
                                download=True,
                                transform=transform)

In [25]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.5, std=0.5)
           )

In [26]:
test_dataset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.5, std=0.5)
           )

In [27]:
## step 7 converting our datasets into dataloaders
## This means breaking dataset into batches or minibatches
## Computationally efficient ascomputing hardware may not handle 50,000 images
## At a go
## so we break it into 128 images at a time (BATCH_SIZE = 128)
## It gives our neural network more chances to update its gradient per epoch